# Sqlite3

## Introduction

In [1]:
import sqlite3
print(sqlite3.version)
print(sqlite3.sqlite_version)
con = sqlite3.connect('./sqlite.db')
print(type(con))
sqlite3.collections

2.6.0
3.16.2
<class 'sqlite3.Connection'>


<module 'collections' from '/home/markbaum/Python/python36/lib/python3.6/collections/__init__.py'>

## Cursor
데이터 작성하기

In [2]:
# SQL 구문을 호충
con = sqlite3.connect('./sqlite.db')
cursor = con.cursor()

In [3]:
# 테이블 생성 및 내용 2줄 추가 (Python Coding)
cursor.execute("create Table KAKAO (Data text, Open int, High int, Low int, Close int, Volumn int)")
cursor.execute("INSERT INTO KAKAO Values('16.06.03', 97000, 98600, 96900, 98000, 321405)")
cursor.execute("INSERT INTO KAKAO Values('16.06.02', 95000, 94600, 91900, 96000, 121405)")

In [6]:
# 지금까지 수정한 내용을 DB에 반영 (saaving from Python to Sqlite)
con.commit()
con.close()

## Cursor 2
데이터 불러오기

In [8]:
# SQL 구문을 호충
import sqlite3
con = sqlite3.connect('./sqlite.db')
cursor = con.cursor()

In [9]:
# Row 단위로 데이터 호출
cursor.execute("SELECT * From KAKAO")
cursor.fetchone()

In [13]:
# Row 전체를 불러오기
cursor.execute("SELECT * From KAKAO") # 테이블 커서를 초기화 (맨 위로 회귀)
cursor.fetchall()

[('16.06.03', 97000, 98600, 96900, 98000, 321405),
 ('16.06.02', 95000, 94600, 91900, 96000, 121405)]

## Indexing and Slicing
데이터 가공하기

In [14]:
# Row 전체를 불러온 뒤 indexing
# 한 테이블의 데이터를 자유자재로 타루기에 용이하다
cursor.execute("SELECT * From KAKAO") # 테이블 커서를 초기화 (맨 위로 회귀)
kakao = cursor.fetchall()
kakao[0][0]  # [row] [column]

'16.06.03'

In [16]:
kakao[0][1:]

(97000, 98600, 96900, 98000, 321405)

## Pandas 
by sqlite3
- https://stackoverflow.com/questions/24189150/pandas-writing-dataframe-to-other-postgresql-schema

### Writing Data
by Pandas

In [20]:
import pandas as pd
from pandas import Series, DataFrame
raw_data = {'col0':[1,2,3,4], 'col1':[10,20,30,40], 'col2':[100,200,300,400]}
df = DataFrame(raw_data); df

,col0,col1,col2
0,1,10,100
1,2,20,200
2,3,30,300
3,4,40,400


In [22]:
import sqlite3
con = sqlite3.connect("./sqlite.db")
df.to_sql('test',con)  # Pandas 에서 바로 연결되어서 저장

### Reading Data
by Pandas

In [26]:
df = pd.read_sql("Select * from KAKAO", con, index_col=None)
df  # Index 값이 추가된 테이블이 생성

,Data,Open,High,Low,Close,Volumn
0,16.06.03,97000,98600,96900,98000,321405
1,16.06.02,95000,94600,91900,96000,121405


In [27]:
df = pd.read_sql("Select * from test", con, index_col=None)
df  # Index 값이 추가된 테이블이 생성

,index,col0,col1,col2
0,0,1,10,100
1,1,2,20,200
2,2,3,30,300
3,3,4,40,400


### Sqlite with Stock Data
by Pandas

In [13]:
from pandas_datareader import get_data_google
df = get_data_google("KRX:005930", '2010-1-1', '2017-7-10')
print(df.head())

                Open      High       Low     Close  Volume
Date                                                      
2010-01-04  803000.0  809000.0  800000.0  809000.0  233324
2010-01-05  826000.0  829000.0  815000.0  822000.0  531612
2010-01-06  829000.0  841000.0  826000.0  841000.0  452442
2010-01-07  841000.0  841000.0  813000.0  813000.0  428015
2010-01-08  820000.0  821000.0  806000.0  821000.0  291154


In [14]:
# Connect to Write Sqlite3
con = sqlite3.connect('./sqlite.db')
df.to_sql('삼성전자', con, if_exists = 'replace') # 기존 자료를 지우고 새롭게 저장

    if_exists = 
    - fail: (기존 자료가 있으면 저장하지 않는다) If table exists, do nothing.
    - replace: (지우고 새롭게 저장) If table exists, drop it, recreate it, and insert data.
    - append: (내용을 추가) If table exists, insert data. Create if does not exist.

In [6]:
# Connect to Read Sqlite3
read_df = pd.read_sql("select * from '삼성전자'",con ,index_col='Date')
print(read_df.tail(5))

                          Open       High        Low      Close  Volume
Date                                                                   
2017-07-07 00:00:00  2387000.0  2406000.0  2381000.0  2393000.0  160473
2017-07-10 00:00:00  2425000.0  2445000.0  2416000.0  2433000.0  211575
2017-07-11 00:00:00  2432000.0  2450000.0  2414000.0  2450000.0  191701
2017-07-12 00:00:00  2450000.0  2500000.0  2442000.0  2494000.0  188641
2017-07-13 00:00:00  2504000.0  2547000.0  2502000.0  2528000.0  308691


### Appending the Data
by Pandas
- 새로운 데이터만 추가해보기
https://www.ryanbaumann.com/blog/2016/4/30/python-pandas-tosql-only-insert-new-rows

In [25]:
from pandas_datareader import get_data_google
df = get_data_google("KRX:005930", '2017-7-10', '2017-7-14')
print(df.head())

                 Open       High        Low      Close  Volume
Date                                                          
2017-07-10  2425000.0  2445000.0  2416000.0  2433000.0  211575
2017-07-11  2432000.0  2450000.0  2414000.0  2450000.0  191701
2017-07-12  2450000.0  2500000.0  2442000.0  2494000.0  188641
2017-07-13  2504000.0  2547000.0  2502000.0  2528000.0  308691


In [16]:
# Connect to Write Sqlite3
con = sqlite3.connect('./sqlite.db')
df.to_sql('삼성전자', con, if_exists = 'append')

In [28]:
# Connect to Read Sqlite3
read_df = pd.read_sql("select * from '삼성전자'",con , index_col='Date')
print(read_df.tail(45))

                          Open       High        Low      Close  Volume
Date                                                                   
2017-06-07 00:00:00  2325000.0  2325000.0  2262000.0  2265000.0  243638
2017-06-08 00:00:00  2250000.0  2279000.0  2250000.0  2258000.0  274158
2017-06-09 00:00:00  2284000.0  2322000.0  2280000.0  2305000.0  227782
2017-06-12 00:00:00  2271000.0  2280000.0  2257000.0  2269000.0  217181
2017-06-13 00:00:00  2257000.0  2281000.0  2257000.0  2270000.0  167722
2017-06-14 00:00:00  2290000.0  2303000.0  2262000.0  2268000.0  194743
2017-06-15 00:00:00  2284000.0  2296000.0  2259000.0  2284000.0  182884
2017-06-16 00:00:00  2275000.0  2297000.0  2273000.0  2279000.0  301134
2017-06-19 00:00:00  2279000.0  2328000.0  2278000.0  2328000.0  212538
2017-06-20 00:00:00  2362000.0  2407000.0  2361000.0  2407000.0  296577
2017-06-21 00:00:00  2387000.0  2406000.0  2374000.0  2374000.0  192556
2017-06-22 00:00:00  2398000.0  2404000.0  2386000.0  2398000.0 